In [1]:
%matplotlib inline
from six.moves.urllib.request import urlretrieve
import zipfile
import numpy as np
import pandas as pd
import os
import time
import random
import tensorflow as tf
from matplotlib import pylab
from scipy.sparse import lil_matrix

2022-09-23 17:27:44.222379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-23 17:27:44.222416: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from os import listdir
from operator import itemgetter
all_files=listdir('/home/slisowski/Portfolio/wig20_sentiment/translated_files_prod/embed/')

#We have to sort files by index number, but this is string so first we need to extract index as integer
file_idx=[]
for file in all_files:
    idx=file.split('_')[2]
    file_idx.append(int(idx))
file_idx

files_idx=zip(all_files, file_idx)
sorted_tuples=sorted(tuple(files_idx),key=itemgetter(1))
sorted_list_files=[]
for tup in sorted_tuples:
    dir_path='/home/slisowski/Portfolio/wig20_sentiment/translated_files_prod/embed/'
    file_path=dir_path+tup[0]
    sorted_list_files.append(file_path)
    

    
news_df=pd.concat(map(pd.read_csv, sorted_list_files), ignore_index=True)

In [3]:
news_df

,Unnamed: 0,company,news_date,news_text
0,0,ASSECO-POLAND,2022-09-16 08:48:05,Thursday on the Polish stock exchange was anot...
1,1,ASSECO-POLAND,2022-09-09 15:37:42,"During the video conferencing, the president o..."
2,2,ASSECO-POLAND,2022-09-06 17:22:00,"""This year, and the whole future Asseco Poland..."
3,3,ASSECO-POLAND,2022-09-02 12:29:08,Given the fatal performance of both WIG20 and ...
4,4,ASSECO-POLAND,2022-09-01 10:35:52,Improving the results of the IT group was poss...
...,...,...,...,...
12597,295,WIG,2020-03-03 11:42:38,Increase in the value of trading in shares und...
12598,296,WIG,2020-03-02 10:22:36,Delayed Coronavir's effect - this is how a com...
12599,297,WIG,2020-02-28 12:18:54,During the turmoil associated with the Coronav...
12600,298,WIG,2020-02-05 09:14:13,"For years, we have been conducting original st..."


In [4]:
tweets_df=pd.read_csv('/home/slisowski/Portfolio/tweets_news_sentiment/tweets_labelled.csv', sep=";")

In [5]:
news_headlines_df=pd.read_csv('/home/slisowski/Portfolio/tweets_news_sentiment/all-data.csv', encoding="Windows-1252", names=["sentiment","text"])

In [6]:
skipgrams_corpus=pd.concat([news_df['news_text'], tweets_df['text'], news_headlines_df['text']])

In [8]:
import unicodedata
from nltk.stem import WordNetLemmatizer
import re
import preprocessor as p
from bs4 import BeautifulSoup
import demoji
import contractions
from nltk.tokenize import word_tokenize


def clean_news_skipgrams(text):
    
    #remove emoji
    text=demoji.replace(text,' ')
    # use tweeter preprocessor to clean news
    text=p.clean(text)
    #remove html tags
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    text=contractions.fix(text)
    
    
    #remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
   
   
    #remove numbers
    text = re.sub("[^a-zA-Z]", " ", text)
    #tokenize text
    text=text.lower()
    
    list_of_words=word_tokenize(text)
    #words=[]
    
    text = ' '.join(list_of_words)
    
    
    
    return text

In [9]:
skipgrams_words=[]
for news in skipgrams_corpus.tolist():
    
    text=clean_news_skipgrams(news)
    skipgrams_words.append(text)

In [12]:
duplicate_sentences=[news for news in skipgrams_words if skipgrams_words.count(news)>1]
    

In [14]:
len(duplicate_sentences)

6927

In [15]:
len(skipgrams_words)

22448

In [16]:
skipgrams_words=list(set(skipgrams_words))

In [17]:
len(skipgrams_words)

17749

In [18]:
news_stories=skipgrams_words

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(
     split=' '
)

tokenizer.fit_on_texts(news_stories)
print("Data fitted on the tokenizer")

Data fitted on the tokenizer


In [20]:
n_vocab = len(tokenizer.word_index.items()) + 1
print(f"Vocabulary size: {n_vocab}")

print("\nWords at the top")
print('\t', dict(list(tokenizer.word_index.items())[:10]))
print("\nWords at the bottom")
print('\t', dict(list(tokenizer.word_index.items())[-10:]))

Vocabulary size: 20783

Words at the top
	 {'the': 1, 'of': 2, 'in': 3, 'to': 4, 'and': 5, 'a': 6, 'for': 7, 'on': 8, 'is': 9, 'will': 10}

Words at the bottom
	 {'airvana': 20773, 'femto': 20774, 'harmonization': 20775, 'dycom': 20776, 'dy': 20777, 'obstacle': 20778, 'economically': 20779, 'saarijarvi': 20780, 'fisker': 20781, 'commercialized': 20782}


In [21]:
news_sequences = tokenizer.texts_to_sequences(news_stories)
news_sequences=[x for x in news_sequences if len(x)>=2]

In [22]:
sample_word_ids = news_sequences[0][:5]
sample_phrase = ' '.join([tokenizer.index_word[wid] for wid in sample_word_ids])
print(f"Sample phrase: {sample_phrase}")
print(f"Sample word IDs: {sample_word_ids}\n")

window_size = 1 # How many words to consider left and right.

inputs, labels = tf.keras.preprocessing.sequence.skipgrams(
    sample_word_ids, 
    vocabulary_size=n_vocab, 
    window_size=window_size, negative_samples=1.0, shuffle=False,
    categorical=False, sampling_table=None, seed=None
)


print("Sample skip-grams")

for inp, lbl in zip(inputs, labels):
    print(f"\tInput: {inp} ({[tokenizer.index_word[wi] for wi in inp]}) / Label: {lbl}")

Sample phrase: dnp dinopl opened new stores
Sample word IDs: [1304, 1122, 805, 41, 320]

Sample skip-grams
	Input: [1304, 1122] (['dnp', 'dinopl']) / Label: 1
	Input: [1122, 1304] (['dinopl', 'dnp']) / Label: 1
	Input: [1122, 805] (['dinopl', 'opened']) / Label: 1
	Input: [805, 1122] (['opened', 'dinopl']) / Label: 1
	Input: [805, 41] (['opened', 'new']) / Label: 1
	Input: [41, 805] (['new', 'opened']) / Label: 1
	Input: [41, 320] (['new', 'stores']) / Label: 1
	Input: [320, 41] (['stores', 'new']) / Label: 1
	Input: [1122, 5171] (['dinopl', 'aug']) / Label: 0
	Input: [805, 918] (['opened', 'assigned']) / Label: 0
	Input: [1304, 12373] (['dnp', 'amplifiers']) / Label: 0
	Input: [41, 11221] (['new', 'imv']) / Label: 0
	Input: [320, 6058] (['stores', 'cells']) / Label: 0
	Input: [1122, 2055] (['dinopl', 'papers']) / Label: 0
	Input: [805, 1757] (['opened', 'abroad']) / Label: 0
	Input: [41, 5519] (['new', 'seat']) / Label: 0


In [23]:
inputs, labels = tf.keras.preprocessing.sequence.skipgrams(
    sample_word_ids, 
    vocabulary_size=len(tokenizer.word_index.items())+1, 
    window_size=window_size, negative_samples=0, shuffle=False,    
)

inputs, labels = np.array(inputs), np.array(labels)

negative_sampling_candidates, true_expected_count, sampled_expected_count = tf.random.log_uniform_candidate_sampler(
    # A true context word that appears in the context of the target
    true_classes=inputs[:1,1:], # [b, 1] sized tensor
    num_true=1, # number of true words per example
    num_sampled=10,
    unique=True,
    range_max=n_vocab,            
    name="negative_sampling"
)

print(f"Positive sample: {inputs[:1,1:]}")
print(f"Negative samples: {negative_sampling_candidates}")
print(f"true_expected_count: {true_expected_count}")
print(f"sampled_expected_count: {sampled_expected_count}")

Positive sample: [[1122]]
Negative samples: [   0    4   59 6414  648   68  924  284 1547    5]
true_expected_count: [[0.00098436]]
sampled_expected_count: [5.4840094e-01 1.8420938e-01 1.8137142e-02 1.7244756e-04 1.7021832e-03
 1.5805339e-02 1.1948386e-03 3.8685752e-03 7.1428146e-04 1.5792997e-01]


2022-09-23 17:40:56.480279: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-23 17:40:56.480310: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-23 17:40:56.480330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (slawomir): /proc/driver/nvidia/version does not exist
2022-09-23 17:40:56.481899: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [24]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(n_vocab, sampling_factor=1e-05)

print(sampling_table)

[0.00315225 0.00315225 0.00547597 ... 1.         1.         1.        ]


In [25]:
def skip_gram_data_generator(sequences, window_size, batch_size, negative_samples, vocab_size, seed=None):
    
    rand_sequence_ids = np.arange(len(sequences))                    
    np.random.shuffle(rand_sequence_ids)


    for si in rand_sequence_ids:
        
        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequences[si], 
            vocabulary_size=vocab_size, 
            window_size=window_size, 
            negative_samples=0.0, 
            shuffle=False,
            sampling_table=sampling_table,
            seed=seed
        )
        
        targets, contexts, labels = [], [], []
        
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(tf.constant([context_word], dtype="int64"), 1)
            
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
              true_classes=context_class,
              num_true=1,
              num_sampled=negative_samples,
              unique=True,
              range_max=vocab_size,              
              name="negative_sampling")

            # Build context and label vectors (for one target word)
            context = tf.concat(
                [tf.constant([context_word], dtype='int64'), negative_sampling_candidates],
                axis=0
            )
            
            label = tf.constant([1] + [0]*negative_samples, dtype="int64")

            # Append each element from the training example to global lists.
            
            targets.extend([target_word]*(negative_samples+1))
            contexts.append(context)
            labels.append(label)
        if (len(contexts)==0) | (len(targets)==0) | (len(labels)==0):
            continue

        contexts, targets, labels = np.concatenate(contexts), np.array(targets), np.concatenate(labels)
        
        assert contexts.shape[0] == targets.shape[0]
        assert contexts.shape[0] == labels.shape[0]
        
        # If seed is not provided generate a random one
        if not seed:
            seed = random.randint(0, 10e6)
            
        np.random.seed(seed)
        np.random.shuffle(contexts)
        np.random.seed(seed)
        np.random.shuffle(targets)
        np.random.seed(seed)
        np.random.shuffle(labels)
        
    
        for eg_id_start in range(0, contexts.shape[0], batch_size):            
            yield (
                targets[eg_id_start: min(eg_id_start+batch_size, targets.shape[0])], 
                contexts[eg_id_start: min(eg_id_start+batch_size, contexts.shape[0])]
            ), labels[eg_id_start: min(eg_id_start+batch_size, labels.shape[0])]


news_skip_gram_gen = skip_gram_data_generator(
    news_sequences, 4, 10, 5, n_vocab
)

for btc, bl in news_skip_gram_gen:
    
    print(btc)
    print(bl)
    
    break

(array([10586,   348, 10586,  7952,   348,  6730,   348, 10586, 10587,
       10587]), array([  11,    3,   22,   28,    4,    2,  129,    5, 6711,    5]))
[1 1 0 0 0 0 0 0 0 0]


In [31]:
batch_size = 4096 # Data points in a single batch

embedding_size = 128 # Dimension of the embedding vector.

window_size=1 # We use a window size of n on either side of target word
negative_samples = 4 # Number of negative samples generated per example

epochs = 10 # Number of epochs to train for

# We pick a random validation set to sample nearest neighbors
valid_size = 16 # Random set of words to evaluate similarity on.
# We sample valid datapoints randomly from a large window without always being deterministic
valid_window = 250

# When selecting valid examples, we select some of the most frequent words as well as
# some moderately rare words as well
np.random.seed(54321)
random.seed(54321)

valid_term_ids = np.array(random.sample(range(valid_window), valid_size))
valid_term_ids = np.append(
    valid_term_ids, random.sample(range(1000, 1000+valid_window), valid_size),
    axis=0
)


In [27]:
import tensorflow.keras.backend as K

K.clear_session()

# Inputs - skipgrams() function outputs target, context in that order
# we will use the same order
input_1 = tf.keras.layers.Input(shape=(), name='target')
input_2 = tf.keras.layers.Input(shape=(), name='context')

# Two embeddings layers are used one for the context and one for the target
context_embedding_layer = tf.keras.layers.Embedding(
    input_dim=n_vocab, output_dim=embedding_size, name='context_embedding'
)
target_embedding_layer = tf.keras.layers.Embedding(
    input_dim=n_vocab, output_dim=embedding_size, name='target_embedding'
)

# Lookup outputs of the embedding layers
target_out = target_embedding_layer(input_1)
context_out = context_embedding_layer(input_2)

# Computing the dot product between the two 
out = tf.keras.layers.Dot(axes=-1)([context_out, target_out])

# Defining the model
skip_gram_model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=out, name='skip_gram_model')

# Compiling the model
skip_gram_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer='adam'
)

skip_gram_model.summary()

Model: "skip_gram_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 context (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 target (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 context_embedding (Embedding)  (None, 128)          2660224     ['context[0][0]']                
                                                                                                  
 target_embedding (Embedding)   (None, 128)          2660224     ['target[0][0]']                 
                                                                                    

In [28]:
class ValidationCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, valid_term_ids, model_with_embeddings, tokenizer):
        
        self.valid_term_ids = valid_term_ids
        self.model_with_embeddings = model_with_embeddings
        self.tokenizer = tokenizer
        
        super().__init__()
        
    def on_epoch_end(self, epoch, logs=None):
        """ Validation logic """
                
        # We will use context embeddings to get the most similar words
        # Other strategies include: using target embeddings, mean embeddings after avaraging context/target
        embedding_weights = self.model_with_embeddings.get_layer("context_embedding").get_weights()[0]
        normalized_embeddings = embedding_weights / np.sqrt(np.sum(embedding_weights**2, axis=1, keepdims=True))
        
        # Get the embeddings corresponding to valid_term_ids
        valid_embeddings = normalized_embeddings[self.valid_term_ids, :]
        
        # Compute the similarity between valid_term_ids and all the embeddings
        # V x d (d x D) => V x D
        top_k = 5 # Top k items will be displayed
        similarity = np.dot(valid_embeddings, normalized_embeddings.T)
        
        # Invert similarity matrix to negative
        # Ignore the first one because that would be the same word as the probe word
        similarity_top_k = np.argsort(-similarity, axis=1)[:, 1: top_k+1]
                
        # Print the output
        for i, term_id in enumerate(valid_term_ids):
            
            similar_word_str = ', '.join([self.tokenizer.index_word[j] for j in similarity_top_k[i, :] if j >= 1])
            print(f"{self.tokenizer.index_word[term_id]}: {similar_word_str}")
        
        print('\n')

In [32]:
skipgram_validation_callback = ValidationCallback(valid_term_ids, skip_gram_model, tokenizer)

for ei in range(epochs):
    
    print(f"Epoch: {ei+1}/{epochs} started")
    
    news_skip_gram_gen = skip_gram_data_generator(
        news_sequences, window_size, batch_size, negative_samples, n_vocab
    )
    
    skip_gram_model.fit(
        news_skip_gram_gen, epochs=1, 
        callbacks=skipgram_validation_callback,        
    )

Epoch: 1/10 started
  17013/Unknown - 850s 50ms/step - loss: 0.4569day: logs, lubiatw, marketing, dip, boasted
term: sized, internal, stays, kaido, kryska
s: shoes, bags, j, whole, biocng
stock: permanent, release, gpw, spread, giving
first: second, third, crucial, engineer, rzeszw
compared: chf, emphasize, fill, jnk, inventors
most: extremely, explosion, suffered, gry, found
investment: whammy, mrl, grad, powerhouse, strives
pkn: lietuva, promotional, wants, poudnie, solidarity
march: watchnow, actual, contrasting, searches, jamal
pekao: santander, peo, deutsche, meets, mied
revenues: dropped, decreased, settled, strongest, muller
sector: dress, gry, banki, imho, daysfibonacci
also: fangdd, entrusted, little, impossible, warranties
data: pushes, initiates, analyzes, crosshairs, moutai
profit: result, loss, corrected, consolidated, profits
ready: ranges, cup, advance, edt, imposing
emission: arabia, sintered, enb, sounds, broadband
yet: rules, decide, vaisala, pointing, rmb
asset: docu

Epoch: 6/10 started
  17021/Unknown - 818s 48ms/step - loss: 0.2768day: seed, night, happen, atmosphere, roc
term: sized, collapse, bronsko, swot, horizon
s: headquartered, atomic, nnen, oriola, recari
stock: floated, seaspine, cannabis, swing, approached
first: second, third, ipko, fourth, shelves
compared: ltm, silent, cx, slipping, totalenergies
most: rtv, lesson, burden, casual, testament
investment: dkr, soh, wisely, successor, sts
pkn: poudnie, wocawska, pacza, siemidarno, shortened
march: february, december, prejudge, rinkuskiai, paneuro
pekao: skiba, polski, czarnecki, peo, pogorzelski
revenues: turnover, baltics, capitalization, influenced, bricks
sector: agri, sdcm, racked, woman, spyback
also: spxanyone, selections, variables, boring, underway
data: pitfalls, shield, disappointing, commentary, debuted
profit: ebit, result, loss, taxes, repetitive
ready: ct, recruit, modernize, gup, omit
emission: stance, refueling, cleaned, invalid, expense
yet: disturbing, lagging, bears, m

In [33]:
def save_embeddings(model, tokenizer, vocab_size, save_dir):
    
    os.makedirs(save_dir, exist_ok=True)
    
    _, words_sorted = zip(*sorted(list(tokenizer.index_word.items()), key=lambda x: x[0])[:vocab_size-1])
        
    words_sorted = [None] + list(words_sorted)
    
    pd.DataFrame(
        model.get_layer("context_embedding").get_weights()[0], 
        index = words_sorted
    ).to_pickle(os.path.join(save_dir, "context_embedding_skipgrams.pkl"))
    
    pd.DataFrame(
        model.get_layer("target_embedding").get_weights()[0], 
        index = words_sorted
    ).to_pickle(os.path.join(save_dir, "target_embedding_skipgrams.pkl"))

    
save_embeddings(skip_gram_model, tokenizer, n_vocab, save_dir='skipgram_embeddings')